In [1]:
import torch
import torch.nn as nn
torch.__version__

'1.0.1.post2'

In [2]:
import torch.nn.functional as F

In [3]:
class Net(nn.Module):
    def __init__(self):
        # nn.Module子类的函数必须在构造函数中执行父类的构造函数
        super(Net,self).__init__()
        # 卷积层中‘1’表示输入图片为单通道，‘6’表示输出通道数，‘3’表示卷积核为3×3
        self.conv1=nn.Conv2d(1,6,3)
        # 线性层，输入字350个特征，输出10个特征
        self.fc1=nn.Linear(1350,10)
    # 正向传播
    def forward(self,x):
        # 卷积=》激活=》池化
        x=self.conv1(x)
        x=F.relu(x)
        x=F.max_pool2d(x,(2,2))
        x=F.relu(x)
        x=x.view(x.size()[0],-1) #reshape,'-1'表示自适应
        x=self.fc1(x)
        return x
net=Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1350, out_features=10, bias=True)
)


In [4]:
for parameters in net.parameters():
    print(parameters)

Parameter containing:
tensor([[[[-0.1519,  0.0142,  0.2089],
          [ 0.0025,  0.1608,  0.1820],
          [-0.0070,  0.3042, -0.1662]]],


        [[[ 0.1403, -0.3036,  0.1329],
          [ 0.3113, -0.1509, -0.2164],
          [-0.2207, -0.3010,  0.1188]]],


        [[[-0.1758, -0.3090, -0.1289],
          [-0.0013, -0.3308, -0.1798],
          [-0.1530,  0.0936, -0.0972]]],


        [[[-0.1587,  0.1828,  0.0315],
          [ 0.1324,  0.0575, -0.0435],
          [ 0.0490,  0.0596,  0.2964]]],


        [[[ 0.0407,  0.0031,  0.2483],
          [-0.1958, -0.1220,  0.1173],
          [ 0.1139, -0.3188, -0.0924]]],


        [[[-0.0283,  0.1543,  0.0170],
          [-0.0187, -0.2998, -0.2954],
          [ 0.0755,  0.0776,  0.2982]]]], requires_grad=True)
Parameter containing:
tensor([-0.1088,  0.0280,  0.2865, -0.2084,  0.1847, -0.2644],
       requires_grad=True)
Parameter containing:
tensor([[ 0.0182, -0.0197, -0.0266,  ..., -0.0162, -0.0270,  0.0185],
        [-0.0170, -0.0030,  0

In [7]:
for name,parameters in net.named_parameters():
    print(name,":",parameters.size())

conv1.weight : torch.Size([6, 1, 3, 3])
conv1.bias : torch.Size([6])
fc1.weight : torch.Size([10, 1350])
fc1.bias : torch.Size([10])


In [8]:
input=torch.randn(1,1,32,32)
out=net(input)
out.size()

torch.Size([1, 10])

In [9]:
net.zero_grad()
out.backward(torch.ones(1,10))

In [10]:
y=torch.arange(0,10).view(1,10).float()
criterion=nn.MSELoss()
loss=criterion(out,y)
print(loss.item())  #loss是个scalar，可以直接使用item获取它的python类型的数值

31.026832580566406


In [11]:
import torch.optim

In [12]:
out=net(input)
criterion=nn.MSELoss()
loss=criterion(out,y)
optimizer=torch.optim.SGD(net.parameters(),lr=0.01) # SGD只需要参数和学习率
optimizer.zero_grad() # 梯度清零
loss.backward()
optimizer.step() #更新参数